### This script performs epoching of the raw eeg data based on three vowel phones |iy|/|uw|/|ae|
The timing of these three vowels are determined by the corresponding syllables.

In [1]:
import os

import numpy as np
from scipy.io import wavfile
from dSPEECH.config import *
import json

2024-08-19 14:32:20 - Start Program

pre_all: Running from CMD.
common_dl.py: Using CUDA.


In [2]:
sid=2
folder=meta_dir+'SEEG'+str(sid)+'/forced_alignment/' #alignment/'
filename=folder+'/syllable_time.npy'
result=np.load(filename, allow_pickle=True).item()

In [28]:
on_beats=result['on_beats']
off_beats=result['off_beats']
on_beats2=result['on_beats2'] # 100
off_beats2=result['off_beats2'] # 100
on_beats_phones=result['on_beats_phones']
off_beats_phones=result['off_beats_phones']

In [39]:
sentences=on_beats2+off_beats2 # 200=100+100
iy=[]
ae=[]
uw=[]
for i in range(100):
    beats=off_beats2[i]+on_beats2[i]
    iy_tmp=[]
    ae_tmp=[]
    uw_tmp=[]
    for syllable in beats:
        key=list(syllable.keys())[0]
        if 'iy' in key:
            iy_tmp.append(syllable)
        elif 'ae' in key:
            ae_tmp.append(syllable)
        elif 'uw' in key:
            uw_tmp.append(syllable)
    iy.append(iy_tmp)
    ae.append(ae_tmp)
    uw.append(uw_tmp)

In [27]:
modality='SEEG'
sid=2 # 1/2
sf=1024
## load epochs and sentences
filename=data_dir+'processed/'+modality+str(sid)+'/'+modality+str(sid)+'-epo.fif'
epochs=mne.read_epochs(filename)
filename2=data_dir+'processed/'+modality+str(sid)+'/sentences.npy'
sentences=np.load(filename2,allow_pickle=True)

# two events: 1 stands for 'TRIG[001]:1' and 2 stands for 'TRIG[001]:1:inserted'
epoch1=epochs['1'].get_data() # (99, 150, 15361)
epoch2=epochs['2'].get_data() # (4, 150, 15361)
epochs=np.concatenate((epoch1,epoch2),axis=0) # (100 trials, 150 channels, 15361 time)


In [14]:
epochs=epochs[:,:,int(5*sf):int(10*sf)]

In [66]:
epoch_iy=[]
epoch_ae=[]
epoch_uw=[]
for i, (iy_sent,ae_sent,uw_sent) in enumerate(zip(iy,ae,uw)):
    for syllable in iy_sent:
        interv=list(syllable.values())[0]
        start=int(sf*interv[0])
        end=int(sf*interv[1])
        epoch_iy.append(epochs[i,:,start:end])
    for syllable in ae_sent:
        interv=list(syllable.values())[0]
        start=int(sf*interv[0])
        end=int(sf*interv[1])
        epoch_ae.append(epochs[i,:,start:end])
    for syllable in uw_sent:
        interv=list(syllable.values())[0]
        start=int(sf*interv[0])
        end=int(sf*interv[1])
        epoch_uw.append(epochs[i,:,start:end])

In [71]:
epoch_iy=[tmp.transpose() for tmp in epoch_iy] # transpose(): when the leading dimmen is different, no problem. but if the tailing dimmen is different, then np.save failed.
epoch_ae=[tmp.transpose() for tmp in epoch_ae]
epoch_uw=[tmp.transpose() for tmp in epoch_uw]
print('Len of iy: '+str(len(epoch_iy))+'; len of ae: '+str(len(epoch_ae))+'; len of uw: '+str(len(epoch_uw))+'.')

Len of iy: 160; len of ae: 171; len of uw: 133.


In [72]:
result={}
result['epoch_iy']=np.array(epoch_iy,dtype=object)
result['epoch_ae']=np.array(epoch_ae,dtype=object)
result['epoch_uw']=np.array(epoch_uw,dtype=object)
filename=data_dir+'processed/'+modality+str(sid)+'/CV/vowel_epoch.npy'
np.save(filename, result)